In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import timeit
from tqdm import tqdm
import pandas as pd
import time
import os 
import seaborn as sns
import random

start_time = time.time()

np.random.seed(1234)
tf.random.set_seed(111)

In [2]:
print (tf.__version__)

2.6.0


### Define PINN neural network

In [3]:
def init_model(number_hidden_layers = 2, num_neurons_per_layer = 50):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(1))
    
    model.add(tf.keras.layers.Dense(num_neurons_per_layer, activation=tf.keras.activations.get('tanh'), 
                                    kernel_initializer='glorot_normal'))
    model.add(tf.keras.layers.Dense(num_neurons_per_layer, activation=tf.keras.activations.get('tanh'), 
                                    kernel_initializer='glorot_normal'))
    
    model.add(tf.keras.layers.Dense(3, activation=tf.keras.activations.get('softmax')))
    
    return model

In [4]:
######################################################################################
# A basic PINN Tensorflow class for solving a continuous-time in-homogeneous Markov chains
class PINN:
    # Initialize the class
    def __init__(self, X_u, Y_u, X_r, model):
        
        ######################################################################################
        # Normalization constants
        self.mu_x, self.sigma_x = X_r.mean(0), X_r.std(0)

        # Normalize inputs
        X_u = (X_u - self.mu_x)/self.sigma_x
        X_r = (X_r - self.mu_x)/self.sigma_x
        self.N_u = X_u.shape[0]
        self.N_r = X_r.shape[0]

        # Store data                
        self.X_u = X_u
        self.Y_u = Y_u
        self.X_r = X_r

        self.Xu_tf = tf.convert_to_tensor(X_u, dtype=tf.float32)
        self.Yu_tf = tf.convert_to_tensor(Y_u, dtype=tf.float32)
        self.Xr_tf = tf.convert_to_tensor(X_r, dtype=tf.float32)
        
        self.model = model
        
        self.iter = 0
        
    def get_r(self):
        with tf.GradientTape(persistent=True) as g:
            g.watch(self.Xr_tf)
            
            u = self.model(self.Xr_tf)
            
            u1 = u[:,0]
            u2 = u[:,1]
            u3 = u[:,2]
            
        u_x_1 = g.jacobian(u1, self.Xr_tf)[0]/self.sigma_x
        u_x_2 = g.jacobian(u2, self.Xr_tf)[0]/self.sigma_x
        u_x_3 = g.jacobian(u3, self.Xr_tf)[0]/self.sigma_x
        
        #calculate residuals
        residual_1 = u_x_1-(-1.286e-4*u1)
        residual_2 = u_x_2-(5.6e-5*u1-1.006e-4*u2)
        residual_3 = u_x_3-(7.26e-5*u1+1.006e-4*u2)
        
        #total residual
        residual = tf.reduce_mean(tf.square(residual_1)) + tf.reduce_mean(tf.square(residual_2)) + \
        tf.reduce_mean(tf.square(residual_3))
        
        loss_1 = tf.reduce_mean(tf.square(residual_1))
        loss_2 = tf.reduce_mean(tf.square(residual_2))
        loss_3 = tf.reduce_mean(tf.square(residual_3))
        
        del g
        
        return loss_1, loss_2, loss_3
    
    @tf.function
    def loss_fn(self, weight = 1):
        u_pred = self.model(self.Xu_tf)     

        # Evaluate loss
        loss_u = tf.reduce_mean(tf.square(self.Yu_tf[:, 0] - u_pred[:, 0]))
        loss_1, loss_2, loss_3 = self.get_r()
        
        return [loss_u, loss_1, loss_2, loss_3]
    
    
    def get_grad(self):
        with tf.GradientTape() as tape:
            loss = self.loss_fn()
        
        g = tape.gradient(loss, self.model.trainable_variables)

        return loss, g
    
    def get_grad_by_PCG(self):
        with tf.GradientTape() as tape:
            loss = self.loss_fn()
            
            assert type(loss) is list
            loss = tf.stack(loss)
            tf.random.shuffle(loss)

            grads_task = tf.vectorized_map(lambda x: tf.concat([tf.reshape(grad, [-1,]) 
                                for grad in tape.gradient(x, self.model.trainable_variables)
                                if grad is not None], axis=0), loss)
        
        num_tasks = len(loss)

        # Compute gradient projections.
        def proj_grad(grad_task):
            for k in range(num_tasks):
                inner_product = tf.reduce_sum(grad_task*grads_task[k])
                proj_direction = inner_product / tf.reduce_sum(grads_task[k]*grads_task[k])
                grad_task = grad_task - tf.minimum(proj_direction, 0.) * grads_task[k]
            return grad_task

        proj_grads_flatten = tf.vectorized_map(proj_grad, grads_task)

        # Unpack flattened projected gradients back to their original shapes.
        proj_grads = []
        for j in range(num_tasks):
            start_idx = 0
            for idx, var in enumerate(self.model.trainable_variables):
                grad_shape = var.get_shape()
                flatten_dim = np.prod([grad_shape.dims[i].value for i in range(len(grad_shape.dims))])
                proj_grad = proj_grads_flatten[j][start_idx:start_idx+flatten_dim]
                proj_grad = tf.reshape(proj_grad, grad_shape)
                if len(proj_grads) < len(self.model.trainable_variables):
                    proj_grads.append(proj_grad)
                else:
                    proj_grads[idx] += proj_grad               
                start_idx += flatten_dim

        grads_and_vars = list(zip(proj_grads, self.model.trainable_variables))
        
        del tape
        return loss, proj_grads
    
    def callback(self):
        if self.iter % 5 == 0:
            print('Iteration {:05d}: loss = {}'.format(self.iter, ','.join(map(str, self.current_loss))))
        self.iter += 1
    
    def train(self, N, optimizer, method):
        """This method performs a gradient descent type optimization."""
        
        @tf.function
        def train_step():
            if method == 'original':
                loss, grad_theta = self.get_grad()
                
            if method == 'PCG_gradient':
                loss, grad_theta = self.get_grad_by_PCG()
            
            # Perform gradient descent step
            optimizer.apply_gradients(zip(grad_theta, self.model.trainable_variables))
            return loss
        
        for i in range(N):
            
            loss = train_step()
            self.current_loss = tf.convert_to_tensor(loss).numpy()
            self.callback()

In [5]:
######################################################################################
# Number of training data
N_u = 1                          # Boundary condition data on u(x)  
N_r = 1000                        # Number of collocation points for minimizing the PDE residual
lb  = np.array([0.0])            # Left boundary of the domain
ub  = np.array([60000.0])        # Right boundary of the domain

# Generate training data
x_u = np.array([[0]])  ##TZ
y_u = np.array([[1,0,0]])   ##TZ                    # Solution at boundary points (dimension N_u x 1)
x_r = np.linspace(lb, ub, N_r)     # Location of collocation points (dimension N_r x 1)

In [6]:
######################################################################################
# Test data for validating the model predictions
n_star = 5000+1
x_star = np.linspace(lb, ub, n_star) #N_star = x_star.shape[0] 

In [7]:
PINN_solver = PINN(x_u, y_u, x_r, init_model())
initial_weights = PINN_solver.model.get_weights()
#initial_weights

In [8]:
optim = tf.keras.optimizers.Adam()
PINN_solver.train(N=20, optimizer=optim, method = 'original')

Iteration 00000: loss = 0.66330856,2.0703956e-09,2.1764046e-10,3.311472e-09
Iteration 00005: loss = 0.4044222,2.0667874e-09,1.8826952e-10,3.4245322e-09
Iteration 00010: loss = 0.17123163,2.6809184e-09,3.3949543e-10,3.557772e-09
Iteration 00015: loss = 0.05464825,3.6558965e-09,6.4399047e-10,3.6765484e-09


In [9]:
PINN_solver = PINN(x_u, y_u, x_r, init_model())
PINN_solver.model.set_weights(initial_weights)

In [10]:
#PINN_solver.model.get_weights()

In [ ]:
optim = tf.keras.optimizers.Adam()
PINN_solver.train(N=20, optimizer=optim, method = 'PCG_gradient')

Iteration 00000: loss = 0.66330856,2.0703956e-09,2.1764046e-10,3.311472e-09
Iteration 00005: loss = 0.40704948,2.0526476e-09,1.7052787e-10,3.327413e-09
Iteration 00010: loss = 0.19135009,2.5280582e-09,2.5054064e-10,3.2718956e-09


In [ ]:
sns.set(font_scale=1.6)